<a href="https://colab.research.google.com/github/yeaeunJi/deep_learning-/blob/main/RNN%EC%9D%84_%EC%82%AC%EC%9A%A9%ED%95%9C_%EB%AC%B8%EC%9E%A5_%EC%83%9D%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 말뭉치(코퍼스)를 사용해 학습한 언어 모델을 이용해 새로운 문장을 생성

## 언어 모델을 사용한 문장 생성
### RNN을 사용한 문장 생성 순서
- 언어 모델 : 현재까지 주어진 단어들에서 다음에 출현하는 단의 확률분포를 출력
- 언어 모델로 선택된 단어를 선택하는 방법
  - 1. 확률이 가장 높은 단어를 선택(결과가 일정하게 정해짐)
  - 2. 각 후보 단어의 확률에 맞게 선택하는 것(결과가 다르게 정해질 수 있음)

- 생성 문장의 다양성을 위해서 확률적으로 선택하는 방법으로 구현 연습

- 언어 모델을 통해 생성한 문장은 훈련 데이터에 존재하지 않고, 새로 생성된 문장임
  - 언어 모델은 훈련 데이터를 암기하는 것이 아닌 사용된 단어의 정렬 패턴을 학습한 것임

### 문장 생성 구현


In [1]:
# 구글 드라이브에 있는 사용자 정의 모듈을  colab에서 사용하기 위한 사용자 설정
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/deep_learning2/

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/deep_learning2


In [2]:
# Rnnlm 클래스를 상속받아 RnnlmGen 클래스 생성
import numpy as np

from common.functions import softmax
from ch06.rnnlm import Rnnlm
from ch06.better_rnnlm import BetterRnnlm

class RnnlmGen(Rnnlm) : # Rnnlm을 상속받음

  # 문장을 생성하는 메서드
  # ========== 입력 파라미터 ========== 
  # start_id - 최초로 주는 단어의 ID
  # sample_size - 샘플링하는 단어의 수
  # skip_ids - 단어 Id의 리스트로 이에 속한 단어 ID는 샘플링하지 않음. PTB에 있는 <unk>나 전처리된 단어를 샘플링하지 않는 용도
  def generate(self, start_id, skip_ids=None, sample_size=100) :
    word_ids = [start_id]

    x = start_id
    while len(word_ids) < sample_size : 
      x = np.array(x).reshape(1,1) # model.predict()에서 미니배치 처리하므로 2차원 배열로 성형(reshape)
      score = self.predict(x) # 각 단어의 정규화되기 전 점수(score) 출력
      p = softmax(score.flatten()) # 소프트맥스 함수로 정규화하여 확률분포 P를 얻음

      sampled = np.radnom.choice(len(p), size=1, p=p) # 확률 분포를 바탕으로 다음 단어 샘플링
      if (skip_ids is None) or (sampled not in skip_ids) :
        x = sampled
        word_ids.append(int(x))

    return word_ids


In [3]:
# 위에서 구현한 RnnlmGen 클래스를 사용하여 문장 생성
# 아무런 학습도 수행하지 않은 상태(가중치 매개변수는 무작위 초깃값인 상태)에서 문장을 생성
from ch07.rnnlm_gen import RnnlmGen
from dataset import ptb

corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
corpus_size = len(corpus)

model = RnnlmGen()
# model.load_params('ch06/Rnnlm.pkl')

# 시작(start) 문자와 skip 문자 설정
start_word = 'you'
start_id = word_to_id[start_word]
skip_words = ['N', '<unk>', '$']
skip_ids = [word_to_id[w] for w in skip_words]

# 문자 생성
word_ids = model.generate(start_id, skip_ids)
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace(' <eos>', '.\n')
print(txt) # 모델의 가중치 초깃값으로 무작위한 값을 사용하여 의미 없는 문장이 생성됨

you enemies holt vigorous fm banc rooted workstations maximum rapid encourages belgium giorgio object dick clear parent touch collateralized speculative wanted oils error teams pall plane bench title newest earmarked banco avondale contracted skf bikers revamping straight main hughes statement measured agreed \*\* heroes dramatically convert audio eating questioning china dlj agricultural conduct a. interpret prior showroom alvin ge banks garage williams phones maxicare lenses collect yard nicaragua slowed wildlife looking contend aer credibility r.h. comptroller examples terrorist collective later level finger mo. arab efforts heller daiwa kume mergers generates pleased insufficient entity montreal brunt inefficient are sponsored backgrounds rattled


In [4]:
# 위에서 구현한 RnnlmGen 클래스를 사용하여 문장 생성
# 학습을 통해 나온 가중치를 불러와 학습을 수행
from ch07.rnnlm_gen import RnnlmGen
from dataset import ptb

corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
corpus_size = len(corpus)

model = RnnlmGen()
model.load_params('ch06/Rnnlm.pkl')

# 시작(start) 문자와 skip 문자 설정
start_word = 'you'
start_id = word_to_id[start_word]
skip_words = ['N', '<unk>', '$']
skip_ids = [word_to_id[w] for w in skip_words]

# 문자 생성
word_ids = model.generate(start_id, skip_ids)
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace(' <eos>', '.\n')
print(txt) # 첫번째로 수행한 결과보다는 괜찮은 결과가 생성되었지만 개선할 여지가 존재

you 'd be able to targeted buyers.
 the thrift confirmed the federal insurance giant walked manufacturer 's studio on the meeting in transportation that contains two people saw ual shares to need ual corp.
 the republicans plan but in the savings-and-loan dealings in case of the insurers.
 but most real trucks also adjusters it will encourage full elevators bought by the discretion of by mr. lawson 's indication for fire.
 feedlots will if the biggest business used buying and light sellers usually be used in at least delicious nature said merrill lynch & co. bank plc


### 더 좋은 문장으로 생성하기 위한 개선방안
- 언어 모델을 향상시켜 더 좋은 문장을 생성하도록 개선 r

In [5]:
# BetterRnnlm 클래스를 상속받아 BetterRnnlmGen 클래스 생성
import numpy as np

from common.functions import softmax
from ch06.rnnlm import Rnnlm
from ch06.better_rnnlm import BetterRnnlm

class BetterRnnlmGen(BetterRnnlm) : # Rnnlm을 상속받음

  # 문장을 생성하는 메서드
  # ========== 입력 파라미터 ========== 
  # start_id - 최초로 주는 단어의 ID
  # sample_size - 샘플링하는 단어의 수
  # skip_ids - 단어 Id의 리스트로 이에 속한 단어 ID는 샘플링하지 않음. PTB에 있는 <unk>나 전처리된 단어를 샘플링하지 않는 용도
  def generate(self, start_id, skip_ids=None, sample_size=100) :
    word_ids = [start_id]

    x = start_id
    while len(word_ids) < sample_size : 
      x = np.array(x).reshape(1,1) # model.predict()에서 미니배치 처리하므로 2차원 배열로 성형(reshape)
      score = self.predict(x) # 각 단어의 정규화되기 전 점수(score) 출력
      p = softmax(score.flatten()) # 소프트맥스 함수로 정규화하여 확률분포 P를 얻음

      sampled = np.random.choice(len(p), size=1, p=p) # 확률 분포를 바탕으로 다음 단어 샘플링
      if (skip_ids is None) or (sampled not in skip_ids) :
        x = sampled
        word_ids.append(int(x))

    return word_ids


In [6]:
from dataset import ptb

corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)
corpus_size = len(corpus)

model = BetterRnnlmGen()
model.load_params('ch07/BetterRnnlm.pkl')

# 시작(start) 문자와 skip 문자 설정
start_word = 'you'
start_id = word_to_id[start_word]
skip_words = ['N', '<unk>', '$']
skip_ids = [word_to_id[w] for w in skip_words]

# 문자 생성
word_ids = model.generate(start_id, skip_ids)
txt = ' '.join([id_to_word[i] for i in word_ids])
txt = txt.replace(' <eos>', '.\n')
print(txt) 

you have an momentum but they are n't the only thing that 's the world 's new greed.
 conservatives concede that denied problems than thrifts and representatives of computers are n't elliott.
 index-arbitrage trading of wells fargo & co. 's u.s. business is a way for any other holding factories 's long-term paper.
 exchange-rate activity is high margins will be based on defending the transaction which a cash load are fully influenced.
 a survey of its members branch producers normally have tried to expand its comprehensive care of capital gains in the fourth recession.
 most


- 위에서 생성한 언어 모델에게 'the meaning of life is'라는 글을 주고, 이어지는 말을 생성하도록 구현도 가능
- 모델에 ['the', 'meaning', 'of', 'life']를 순서대로 주어 순전파를 수행하고, 그 다음 'is'를 첫 단어로 입력하여 문장을 생성하면 이에 이어지는 문장 생성 가능


## seq2seq
- 언어 데이터, 음성 데이터, 동영상 데이터 모두 시계열 데이터
- 시계열 데이터를 다른 시계열 데이터로 변환하는 기법

### seq2seq의 원리
- Encoder-Decoder 모델이라고도 불리는데, 2개의 모듈인 인코더와 디코더가 등장
  - Encoder : 입력 데이터를 부호화(인코딩)

  * 부호화 : 정보를 어떤 규칙에 따라 변환하는 것

  - Decoder : 인코딩된 데이터를 디코딩(복호화)

  * 복호화 : 인코딩된 정보를 원래의 정보로 되돌리는 것

- 한국어를 영어로 번역하는 것 역시 이러한 과정을 통해 가능
  - 인코더 : '나는 고양이로소이다'를 출발어 문장으로 인코딩
  - 인코딩한 정보를 디코더에 전달
  - 디코더 : 도착어 문장을 생성하되 인코더가 인코딩한 정보에는 번역에 필요한 정보가 조밀하게 응축되어 있어 이를 바탕으로 디코더가 도착어 문장을 생성

- 이러한 과정을 RNN을 통해서 구현 가능

#### Encoder의 처리
- 인코더는 RNN을 사용하여 시계열 데이터(출발어 문장)를 h라는 은닉 상태 벡터로 변환
  - 이 과정을 RNN의 LSTM을 사용하거나 단순 RNN이나 GRU 등도 사용 가능

- LSTM 계층의 마지막 은닉 상태는 인코더가 출력하는 벡터 h로 입력문장을 번역하는데 필요한 정보가 인코딩 됨
- 이때,  h는 고정 길이 벡터이므로 인코딩은 곧 임의 길이의 문장을 고정 길이의 벡터로 변환하는 작업이라고 볼 수 있음

### 시계열 데이터 변환용 장난감 문제
- 시계열 변환 문제의 예로 '더하기'를 살펴봄
- '57+5'를 문자열로 seq2seq로 건네면 62라는 정답을 내도록 학습

* 참고 : 머신러닝을 평가하고자 만든 간단한 문제를 '장난감 문제(toy problem)'이라고 함

- seq2seq로 덧셈을 풀기위해서는 샘플로부터 사용되는 문자의 패턴을 학습하여 규칙을 학습해야 함
- 이 문제를 해결하기 위하여 단어 단위가 아닌 문자 단위로 분할
  - 문자 단위 분할 : 문자별로 리스트 처리

### 가변 길이 시계열 데이터 
- 이러한 덧셈 문제는 샘플마다 데이터의 시간 방향 크기가 다름
  - 예) 628+521 이 들어올지 57+1이 들어올 지 모름
  - 즉, 가변 길이 시계열 데이터를 다룬다는 의미이므로 신경망 학습 시 '미니배치 처리'를 위한 노력이 필요함

- 가변 길이 시계열 데이터를 미니배치로 학습하기 위해 사용할 수 있는 가장 단순한 방법은 패딩(padding)을 사용하는 것
  - 패딩 : 원래의 데이터에 의미 없는 데이터를 채워서 모든 데이터의 길이를 균일하게 맞출 수 있는 기법

- 패딩을 적용하여 데이터 크기를 통일시키면 가변 길이 시계열 데이터도 처리가능하지만 존재하지 않던 문자까지도 seq2seq가처리하게 됨
  - 따라서 seq2seq에 패딩 전용 처리도 추가해야함
  - 디코더에 입력된 데이터가 패딩일 경우 손실의 결과에 반영하지 않도록.
  - 인코더에 입력된 데이터가 패딩일 경우 LSTM 계층이 이전 시각의 입력을 그대로 출력하도록 하여 처리 가능

### 덧셈 데이터셋을 사용

In [7]:
from dataset import sequence

(x_train, t_train), (x_test, t_test) = sequence.load_data('addition.txt', seed=1984)
char_to_id, id_to_char = sequence.get_vocab()

print(x_train.shape, t_train.shape)
print(x_test.shape, t_test.shape)

print(x_train[0])
print(t_train[0])

print(''.join([id_to_char[c] for c in x_train[0]]))
print(''.join([id_to_char[c] for c in t_train[0]]))

(45000, 7) (45000, 5)
(5000, 7) (5000, 5)
[ 3  0  2  0  0 11  5]
[ 6  0 11  7  5]
71+118 
_189 


## seq2seq 구현
- 2개의 RNN 계층을 연결한 신경망을 구현

### Encoder 클래스
- 문자 ID를 문자 벡터로 변환하는 Embedding 계층과  LSTM 계층



In [9]:
# 초기화, 순전화, 역전파를 담당하는 메서드 제공
from common.time_layers import *
class Encoder :
  def __init__(self, vocab_size, wordvec_size, hidden_size) :
    V, D, H = vocab_size, wordvec_size, hidden_size
    rn = np.random.randn

    # Xaiver 초깃값(사비에르 초깃값) : 이전 계층의 노드가 n개 일때 1/root(n)의 분포로 값들로 초기화
    embed_W = (rn(V,D) / 100).astype('f')
    lstm_Wx = (rn(D, 4 * H) / np.sqrt(D)).astype('f') 
    lstm_Wh = (rn(H, 4 * H) / np.sart(H)).astype('f')
    lstm_b = np.zeros(4*H).astype('f')

    self.embed = TimeEmbedding(embed_W)
    self.lstm = TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=False) # Time LSTM 계층이 상태를 유지하지 않음

    self.params = self.embed.params + self.lstm.params
    self.grads = self.embed.grads + self.lstm.grads
    self.hs = None

  def forward(self, xs) :
    xs = self.embed.forward(xs)
    hs = self.lstm.forward(xs)
    self.hs = hs
    return hs[:, -1, :] # Time LSTM 계층의 마지막 시각의 은닉 상태만 출력

  def backward(self, dh) : # dh : LSTM 계층의 마지막 은닉 상태에 대한 기울기로 디코더가 전해준 기울기임
    dhs = np.zeros_like(self.hs) # 원소가 모두 0인 텐서 dhs
    dhs[:, -1, :] = dh # dh를 dhs의 해당 위치에 할당

    dout = self.lstm.backward(dhs)
    dout = self.embed.backward(dout)
    return dout

### Decoder 클래스
- Encoder 클래스가 출력한 h를 받아 다른 문자열을 출력


In [10]:
class Decoder :
  def __init__(self, vocab_size, wordvec_size, hidden_size) :
    V, D, H = vocab_size, wordvec_size, hidden_size
    rn = np.random.randn

    embed_W = (rn(V,D) / 100).astype('f')
    lstm_Wx = (rn(D, 4 * H) / np.sqrt(D)).astype('f')
    lstm_Wh = (rn(H, 4 * H) / np.sqrt(H)).astype('f')
    lstm_b = np.zeros(4 * H).astype('f')
    affine_W = (rn(H, V) / np.sqrt(H)).astype('f')
    affine_b = np.zeros(V).astype('f')

    self.embed = TimeEmbedding(embed_W)
    self.lstm = TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=True)
    self.affine = TimeAffine(affine_W, affine_b)
    self.params, self.grads = [], []

    for layer in (self.embed, self.lstm, self.affine) :
      self.params += layer.params
      self.grads += layer.grads

  def forward(self, xs, h) :
    self.lstm.set_state(h)

    out = self.embed.forward(xs)
    out = self.lstm.forward(out)
    score = self.affine.forward(out)
    return score

  def backward(self, dscore) : # dscore : 위쪽의 SoftmaxWithLoss 계층에서 기울기 dscore를 받음
    dout = self.affine.backward(dscore)
    dout = self.lstm.backward(dout)
    dout = self.embed.backward(dout)
    dh = self.lstm.dh 
    return dh

  # 문장 생성 메소드
  def generate(self, h, start_id, sample_size) :
    sampled = []
    sample_id = start_id
    self.lstm.set_state(h)

    for _ in range(sample_size) :
      x = np.array(sample_id).resyape((1,1))
      out = self.embed.forward(x)
      out = self.lstm.forward(out)
      score = self.affine.forward(out)

      sample_id = np.argmax(score.flatten())
      sampled.append(int(sample_id))

    return sampled

### seq2seq 클래스
- Encoder 클래스와 Deocder 클래스를 연결하고 Time Softmax Wit Loss 계층을 이용하여 손실을 계산


In [12]:
from common.base_model import BaseModel
class Seq2seq(BaseModel) : 
  def __init__(self, vocab_size, wordvec_size, hidden_size) :
    v, D, H = vocab_size, wordvec_size, hidden_size
    self.encoder = Encoder(V, D, H)
    self.decoder = Decoder(V, D, H)
    self.softmax = TimeSoftmaxWithLoss()

    self.params = self.encoder.params + self.decoder.params
    self.grads = self.encdoer.grads + self.decoder.grads

  def forward(self, xs, ts) :
    decoder_xs, decdoer_ts = ts[:, :-1], ts[:, 1:]
    h = self.encoder.foward(xs)
    score = self.decoder.forward(decoder_xs, h)
    loss = self.softmax.forward(score, decoder_ts)
    return loss

  def backward(self, dout=1) :
    dout = self.softmax.backward(dout)
    dh = self.decoder.backward(dout)
    dout = self.encoder.backward(dh)
    return dout

  def generate(self, xs, start_id, sample_size) :
    h = self.encoder.forward(xs)
    sampled = self.decoder.generate(h, start_id, sample_size)
    return sampled

### seq2seq 평가
- 1. 학습 데이터에서 미니배치를 선택
- 2. 미니배치로부터 기울기 계산
- 3. 기울기를 사용하여 매개변수 갱신

In [ ]:
# 매 에폭마다 seq2seq가 테스트 데이터를 풀어(문자열 생성을 수행)하여 학습 중간중간 정답률을 측정
import sys
sys.path.append('..')
import numpy as np
import matplotlib.pyplot as plt
from dataset import sequence
from common.optimizer import Adam
from common.trainer import Trainer
from common.util import eval_seq2seq
from ch07.seq2seq import Seq2seq

%cd /content/drive/MyDrive/deep_learning2/ch07/
from peeky_seq2seq import PeekySeq2seq
%cd /content/drive/MyDrive/deep_learning2/

# 데이터셋 읽기
(x_train, t_train), (x_test, t_test) = sequence.load_data('addition.txt')
char_to_id, id_to_char = sequence.get_vocab()

# 하이퍼파라미터 설정
vocab_size = len(char_to_id)
wordvec_size = 16
hidden_size = 128
batch_size = 128
max_epoch = 25
max_grad = 5.0

# 모델/옵티마이저/트레이너 생성
model = Seq2seq(vocab_size, wordvec_size, hidden_size)
optimizer = Adam()
trainer = Trainer(model, optimizer)

acc_list = []
for epoch in range(max_epoch) :
  trainer.fit(x_train, t_train, max_epoch=1, 
              batch_size=batch_size, max_grad = max_grad)
  
  correct_num = 0
  for i in range(len(x_test)) :
    question, correct = x_test[[i]], t_test[[i]]
    verbose = i < 10
    correct_num += eval_seq2seq(model, question, correct, id_to_char, verbose)

  acc = float(correct_num) / len(x_test)
  acc_list.append(acc)
  print('검증 정확도 %.3f%%'%(acc*100))


## seq2seq 개선
- 학습 속도를 개선하기 위해 효과적인 기법 2가지 

### 입력 데이터 반전(Reverse)
- 입력 데이터의 순서를 반전시켜 학습 진행 속도를 개선함
- 입력 데이터를 반전시키면 학습 진행이 빨라지고 정확도가 향상되는 이유는 직관적으로는 기울기 전파가 원할해지기 때문일 것이라고 생각됨
  - 입력 문장의 첫 부분에서 반전으로 인해 대응하는 변환 후 단어와 가까워지는 경우가 많아지므로 기울기가 더 잘 전해져서 학습 효율이 좋아진다고 생각할 수 있음
  -  입력 데이터를 반전해도 단어 사이의 평균적인 거리는 그대로임

In [ ]:
# 매 에폭마다 seq2seq가 테스트 데이터를 풀어(문자열 생성을 수행)하여 학습 중간중간 정답률을 측정
import sys
sys.path.append('..')
import numpy as np
import matplotlib.pyplot as plt
from dataset import sequence
from common.optimizer import Adam
from common.trainer import Trainer
from common.util import eval_seq2seq
from ch07.seq2seq import Seq2seq

%cd /content/drive/MyDrive/deep_learning2/ch07/
from peeky_seq2seq import PeekySeq2seq
%cd /content/drive/MyDrive/deep_learning2/

# 데이터셋 읽기
(x_train, t_train), (x_test, t_test) = sequence.load_data('addition.txt')
x_trian, x_test = x_train[:, ::-1], x_test[:, ::-1]
char_to_id, id_to_char = sequence.get_vocab()

# 하이퍼파라미터 설정
vocab_size = len(char_to_id)
wordvec_size = 16
hidden_size = 128
batch_size = 128
max_epoch = 25
max_grad = 5.0

# 모델/옵티마이저/트레이너 생성
model = Seq2seq(vocab_size, wordvec_size, hidden_size)
optimizer = Adam()
trainer = Trainer(model, optimizer)

acc_list = []
for epoch in range(max_epoch) :
  trainer.fit(x_train, t_train, max_epoch=1, 
              batch_size=batch_size, max_grad = max_grad)
  
  correct_num = 0
  for i in range(len(x_test)) :
    question, correct = x_test[[i]], t_test[[i]]
    verbose = i < 10
    correct_num += eval_seq2seq(model, question, correct, id_to_char, verbose)

  acc = float(correct_num) / len(x_test)
  acc_list.append(acc)
  print('검증 정확도 %.3f%%'%(acc*100))


### 엿보기(Peeky)
- seq2seq의 Encoder에서는 입력 문장(문제 문장)을 고정 길이 벡터 h로 변환하는데, 이때 h안에는 Decoder에 필요한 정보가 담겨있음
- 위에서 구현한 seq2seq에서는 이러한 h를 최초 시각의 LSTM 계층에서만 사용
- 이러한 정보를 더 잘활용하기 위해 Encoder의 출력 h를 Decoder의 다른 계층에게도 전해줌

- 인코딩된 정보를 디코더의 다른 계층에도 전해주게 되면 LSTM 계층과 Affine 계층에 입력되는 벡터가 2개가 됨
  - 두 벡터가 연결(concatenate)된 것을 의미

In [28]:
# Peeky Decoder
class PeekyDecoder :
  def __init__(self, vocab_size, wordvec_size, hidden_size) :
    V, D, H = vocab_size, wordvec_size, hidden_size
    rn = np.random.randn

    embed_W = (rn(V, D) / 100).astype('f')
    lstm_Wx = (rn(H+D, 4 * H) / np.sqrt(H+D)).astype('f')
    lstm_Wh = (rn(H, 4 * H) / np.sqrt(H)).astype('f')
    lstm_b = np.zeros(4 * H).astype('f')
    affine_W = (rn(H + H, V) / np.sqrt(H+H)).astype('f')
    affine_b = np.zeros(V).astype('f')

    self.embed = TimeEmbedding(embed_W)
    self.lstm = TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=True)
    self.affine = TimeAffine(affine_W, affine_b)

    self.params, self.grads = [], []
    for layer in (self.embed, self.lstm, self.affine) :
      self.params += layer.params
      self.grads += layer.grads
    
    self.cache = None

  def forward(self, xs, b) : 
    N, T = xs.shape
    N, H = h.shape

    self.lstm.set_state(h)

    out = self.embed.forward(xs)
    hs = np.repeat(h, T, axis=0).reshape(N, T, H)
    out = np.concatenate((hs, out), axis=2)

    out = self.lstm.forward(out)
    out = np.concatenate((hs, out), axis=2)

    score = self.affine.forward(out)
    self.cache = H
    return score

In [29]:
from seq2seq import Seq2seq, Encoder

class PeekySeq2seq(Seq2seq) :
  def __init__(self, vocab_size, wordvec_size, hidden_size) :
    V, D, H = vocab_size, wordvec_size, hidden_size
    self.encoder = Encoder(V, D, H)
    self.decoder = PeekyDecoder(V, D, H)
    self.softmax = TimeSoftmaxWithLoss()

    self.params = self.encoder.params + self.decoder.params
    self.grads = self.encoder.grads + self.decoder.grads 

In [30]:
model = PeekySeq2seq(vocab_size, wordvec_size, hidden_size)

- Reverse와 Peekcy를 모두 사용할 경우, 효과적으로 개선될 수 있음
- 어텐션이라는 기술을 사용하면 seq2seq 기법을 극적으로 진화시킬 수 있음
- Peeky 주의 사항
  - Peeky를 사용하게 되면 신경망의 가중치 매개변수가 커져 계산량이 증가되는 핸디캡을 감안항여야 함
  - seq2seq의 정확도는 하이퍼파라미터에 영향을 크게 받으므로 실제 문제에서의 효과는 달라질 수 있음

## seq2seq를 이용하는 애플리케이션
- seq2seq는 한 시계열 데이터를 다른 시계열 데이터로 변환하는 것이므로 이를 다양한 문제에 적용 가능
  - 기계 번역
  - 자동 요약(긴 문장을 짧게 요약된 문장으로 변환)
  - 질의응답(질문을 응답으로 변환)
  - 메일 자동 응답(받은 메일의 문장을 답변 글로 변환)
  - 자연어 외에도 음성이나 영상등에 이용가능
  - 직관적으로는 seq2seq가 적용될 수 없을 것같은 영역에서도 전처리를 통해 입출력 데이터를 가공하면 적용가능해지는 케이스들도 존재

### 챗봇
- 사람과 컴퓨터가 텍스트로 대화를 나누는 프로그램
- 상대의 말을 자신의 말로 변환하는 문제로 생각하여 대화의 텍스트 데이터가 준비되면 그것을 seq2seq로 학습시켜 적용 가능

### 알고리즘 학습
- 위에서 구현해보았던 덧셈보다 더 고차원적인 문제도 처리 가능
- 소스 코드도 문자로 쓰여진 시계열 데이터이므로 그 코드를 입력으로 사용하여 원하는 답과 대조하여 학습시킬 수 있음

### 이미지 캡셔닝
- seq2seq를 통해서 텍스트 외의 데이터도 처리 가능
- 이미지를 문장으로 변환하는 이미지 캡셔닝 기법
- Encoder에서 LSTM 계층을 합성 신경망(CNN) 계층으로 바꿈

- CNN의 최종 출력은 feature map으로 높이, 폭, 채널의 3차원을 출력하기 때문에 이를 Decoder의 LSTM이 처리할 수 있도록 개선하면 됨
  - feature map을 1차원으로 평탄화(flattening)을 한 후 Affine 계층에서 변환하면 됨